In [ ]:
import tkinter as tk
import numpy as np
import pandas as pd

In [ ]:
conditions_list = ["", "<=", ">=", "[ , ]"]

In [3]:
vjump_criteria_df = pd.read_csv("../inputs_outputs/models/vjump_criteria.csv")
vjump_criteria_df["condition"].fillna(value="", inplace=True)
vjump_criteria_df

,frame,criteria_description,condition,min_value,max_value
0,SQUAT POSTURE ANGLES,Elbow angle,>=,160.0,NaN
1,SQUAT POSTURE ANGLES,Knee angle,"[ , ]",75.0,105.0
2,SQUAT POSTURE ANGLES,Shoulder angle,,NaN,NaN
3,SQUAT POSTURE ANGLES,Hip angle,,NaN,NaN
4,JUMP PEAK POSTURE ANGLES,Elbow angle,>=,150.0,NaN
5,JUMP PEAK POSTURE ANGLES,Knee angle,>=,150.0,NaN
6,JUMP PEAK POSTURE ANGLES,Torso leaning angle,<=,NaN,20.0
7,LANDING POSTURE ANGLES,Knee angle,<=,NaN,150.0
8,LANDING POSTURE ANGLES,Torso leaning angle,<=,NaN,10.0


In [4]:
vjump_criteria_dict = {}

for c in vjump_criteria_df.columns:
    vjump_criteria_dict[c] = {}
    for i in vjump_criteria_df.index:
        # print(vjump_criteria_dict)
        vjump_criteria_dict[c][i] = vjump_criteria_df.loc[i, c]
        # print(vjump_criteria_dict)

In [11]:
vjump_criteria_dict

{'frame': {0: 'SQUAT POSTURE ANGLES',
  1: 'SQUAT POSTURE ANGLES',
  2: 'SQUAT POSTURE ANGLES',
  3: 'SQUAT POSTURE ANGLES',
  4: 'JUMP PEAK POSTURE ANGLES',
  5: 'JUMP PEAK POSTURE ANGLES',
  6: 'JUMP PEAK POSTURE ANGLES',
  7: 'LANDING POSTURE ANGLES',
  8: 'LANDING POSTURE ANGLES'},
 'criteria_description': {0: 'Elbow angle',
  1: 'Knee angle',
  2: 'Shoulder angle',
  3: 'Hip angle',
  4: 'Elbow angle',
  5: 'Knee angle',
  6: 'Torso leaning angle',
  7: 'Knee angle',
  8: 'Torso leaning angle'},
 'condition': {0: '>=',
  1: '[ , ]',
  2: '',
  3: '',
  4: '>=',
  5: '>=',
  6: '<=',
  7: '<=',
  8: '<='},
 'min_value': {0: 160.0,
  1: 75.0,
  2: nan,
  3: nan,
  4: 150.0,
  5: 150.0,
  6: nan,
  7: nan,
  8: nan},
 'max_value': {0: nan,
  1: 105.0,
  2: nan,
  3: nan,
  4: nan,
  5: nan,
  6: 20.0,
  7: 150.0,
  8: 10.0}}

In [ ]:
vjump_criteria_dict = {}

for f in vjump_criteria_df["frame"].unique():
    vjump_criteria_dict[f] = {}
    
    temp_df = vjump_criteria_df[vjump_criteria_df["frame"]==f]
    
    for desc in temp_df["criteria_description"].unique():
        
        vjump_criteria_dict[f][desc] = {}
        
        condition = temp_df[temp_df["criteria_description"]==desc]["condition"].tolist()[0]
        min_value = temp_df[temp_df["criteria_description"]==desc]["min_value"].tolist()[0]
        max_value = temp_df[temp_df["criteria_description"]==desc]["max_value"].tolist()[0]
        
        temp_dict = {"condition":condition, "min_value":min_value, "max_value":max_value}
        
        vjump_criteria_dict[f][desc] = temp_dict

In [ ]:
vjump_criteria_dict

In [ ]:
vjump_criteria_df[(vjump_criteria_df["frame"]=="SQUAT POSTURE ANGLES") & (vjump_criteria_df["criteria_description"]=="Elbow angle")]["condition"].tolist()[0]

In [ ]:
vjump_criteria_df[(vjump_criteria_df["frame"]=="SQUAT POSTURE ANGLES") & (vjump_criteria_df["criteria_description"]=="Knee angle")]["condition"].tolist()[0]

In [ ]:
vjump_criteria_dict

In [29]:
def create_input_angles_frame(master, text, prev_condition="", prev_min="", prev_max=""):
    
    frm = tk.Frame(master=master)
    frm.pack(fill=tk.X, expand=True)
    # print(frm.master)
    
    frm_text = tk.Label(frm, text=text, width=15)
    frm_text.pack(side=tk.LEFT, padx=5)
    
    
    var_condition = tk.StringVar()
    var_condition.set(prev_condition)
    
    frm_ent_condition = tk.OptionMenu(frm, var_condition, *conditions_list)
    frm_ent_condition.configure(width=5)
    frm_ent_condition.pack(side=tk.LEFT, padx=5)
    
    
    var_min = tk.StringVar()
    var_min.set(prev_min)
    
    frm_ent_min = tk.Entry(frm, textvariable=var_min, width=8, state=tk.DISABLED)
    if prev_min != "":
        frm_ent_min.configure(state=tk.NORMAL)
    frm_ent_min.pack(side=tk.LEFT, padx=5)
    
    var_max = tk.StringVar()
    var_max.set(prev_max)
    
    frm_ent_max = tk.Entry(frm, textvariable=var_max, width=8, state=tk.DISABLED)
    if prev_max != "":
        frm_ent_max.configure(state=tk.NORMAL)
    frm_ent_max.pack(side=tk.LEFT, padx=5)
    
    
    frm_warning = tk.Label(frm, text="", fg="red", width=25)
    frm_warning.pack(side=tk.LEFT, padx=5)

    
    def conditions_callback(*args):
        
        chosen_condition = var_condition.get()
        
        if chosen_condition=="":
            frm_ent_min.configure(state=tk.DISABLED)
            frm_ent_max.configure(state=tk.DISABLED)
            frm_warning.configure(text="")
        elif chosen_condition==">=":
            frm_ent_min.configure(state=tk.NORMAL)
            frm_ent_max.configure(state=tk.DISABLED)
            min_callback()
        elif chosen_condition=="<=":
            frm_ent_min.configure(state=tk.DISABLED)
            frm_ent_max.configure(state=tk.NORMAL)
            max_callback()
        else:
            frm_ent_min.configure(state=tk.NORMAL)
            frm_ent_max.configure(state=tk.NORMAL)
            min_max_callback()
    
    var_condition.trace("w", conditions_callback)
    
    
    def min_callback(*args):
        
        chosen_min = var_min.get()
        
        try:
            chosen_min = float(chosen_min)
            
            if (chosen_min<0 or chosen_min>180):
                frm_ent_min.configure(fg="red")
                frm_warning.configure(text="Enter decimals between 0-180!")
            else:
                frm_ent_min.configure(fg="green")
                frm_warning.configure(text="")
                
                if frm_ent_max["state"]=="normal":
                    chosen_max = var_max.get()
                    
                    try:
                        chosen_max = float(chosen_max)
                
                        if chosen_min > chosen_max:
                            frm_warning.configure(text="Min value must be <= Max value!")
                        else:
                            frm_warning.configure(text="")
                
                    except:
                        pass
            
        except:
            frm_ent_min.configure(fg="red")
            frm_warning.configure(text="Enter decimals between 0-180!")
    
    var_min.trace("w", min_callback)
    
    
    def max_callback(*args):
        
        chosen_max = var_max.get()
        
        try:
            chosen_max = float(chosen_max)
            
            if (chosen_max<0 or chosen_max>180):
                frm_ent_max.configure(fg="red")
                frm_warning.configure(text="Enter decimals between 0-180!")
            else:
                frm_ent_max.configure(fg="green")
                frm_warning.configure(text="")
                
                if frm_ent_min["state"]=="normal":
                    chosen_min = var_min.get()
                    
                    try:
                        chosen_min = float(chosen_min)
                
                        if chosen_min > chosen_max:
                            frm_warning.configure(text="Min value must be <= Max value!")
                        else:
                            frm_warning.configure(text="")
                
                    except:
                        pass
            
        except:
            frm_ent_max.configure(fg="red")
            frm_warning.configure(text="Enter decimals between 0-180!")
    
    var_max.trace("w", max_callback)
    
    
    def min_max_callback(*args):
        
        chosen_min = var_min.get()
        chosen_max = var_max.get()
        
        try:
            chosen_min = float(chosen_min)
            chosen_max = float(chosen_max)
            
            if (chosen_max<0 or chosen_max>180):
                frm_ent_max.configure(fg="red")
                frm_warning.configure(text="Enter decimals between 0-180!")
            elif (chosen_min<0 or chosen_min>180):
                frm_ent_min.configure(fg="red")
                frm_warning.configure(text="Enter decimals between 0-180!")
            elif chosen_min > chosen_max:
                frm_warning.configure(text="Min value must be <= Max value!")
            else:
                # frm_ent_max.configure(fg="green")
                frm_warning.configure(text="")

        except:
            # frm_ent_max.configure(fg="red")
            frm_warning.configure(text="Enter decimals between 0-180!")
    
    var_max.trace("w", max_callback)

In [30]:
def input_data_to_create_frames(row):
    
    text = row["criteria_description"]
    prev_condition = row["condition"]
    prev_min = row["min_value"]
    prev_max = row["max_value"]
    
    if np.isnan(prev_min):
        prev_min = ""
    else:
        prev_min = str(prev_min)
        
    if np.isnan(prev_max):
        prev_max = ""
    else:
        prev_max = str(prev_max)
    
    if "SQUAT" in row["frame"]:
        create_input_angles_frame(frm_squat, text, prev_condition, prev_min, prev_max)
    elif "JUMP" in row["frame"]:
        create_input_angles_frame(frm_peak, text, prev_condition, prev_min, prev_max)
    elif "LANDING" in row["frame"]:
        create_input_angles_frame(frm_land, text, prev_condition, prev_min, prev_max)

In [31]:
def save_data(row):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-31-d0c8b76222c5>, line 2)

In [32]:
input_window = tk.Tk()

frm_headers = tk.Frame(master=input_window)
frm_headers.pack(fill=tk.X, expand=True)

txt_body_angles = tk.Label(master=frm_headers, text="Body Angles", width=15)
txt_body_angles.pack(side=tk.LEFT, padx=5)

frm_spacer1 = tk.Frame(master=frm_headers, width=6)
frm_spacer1.pack(side=tk.LEFT)

frm_conditions = tk.Frame(master=frm_headers, width=5)
frm_conditions.pack(side=tk.LEFT, padx=5)
txt_conditions = tk.Label(master=frm_conditions, text="Conditions")
txt_conditions.pack()

frm_spacer2 = tk.Frame(master=frm_headers, width=7)
frm_spacer2.pack(side=tk.LEFT)

txt_min_angle = tk.Label(master=frm_headers, text="Min angle (deg)", width=8, wraplength=30)
txt_min_angle.pack(side=tk.LEFT)

txt_max_angle = tk.Label(master=frm_headers, text="Max angle (deg)", width=8, wraplength=30)
txt_max_angle.pack(side=tk.LEFT)

txt_warnings = tk.Label(master=frm_headers, text="Warnings", width=25)
txt_warnings.pack(side=tk.LEFT, padx=5)


frm_squat = tk.Frame(master=input_window, relief=tk.GROOVE, borderwidth=2)
frm_squat.pack(fill=tk.BOTH, expand=True)

txt_squat_posture = tk.Label(master=frm_squat, text="SQUAT POSTURE ANGLES")
txt_squat_posture.pack()


frm_peak = tk.Frame(master=input_window, relief=tk.GROOVE, borderwidth=2)
frm_peak.pack(fill=tk.BOTH, expand=True)

txt_peak_posture = tk.Label(master=frm_peak, text="JUMP PEAK POSTURE ANGLES")
txt_peak_posture.pack()


frm_land = tk.Frame(master=input_window, relief=tk.GROOVE, borderwidth=2)
frm_land.pack(fill=tk.BOTH, expand=True)

txt_land_posture = tk.Label(master=frm_land, text="LANDING POSTURE ANGLES")
txt_land_posture.pack()


for i in vjump_criteria_df.index:
    input_data_to_create_frames(vjump_criteria_df.loc[i])

    
frm_buttons = tk.Frame(master=input_window)
frm_buttons.pack(fill=tk.X, expand=True)
    
but_save = tk.Button(master=frm_buttons, text="Save")#, command=callback)
but_save.pack(side=tk.LEFT, padx=5)

input_window.mainloop()